<a href="https://colab.research.google.com/github/Chromis07/dataStudy/blob/main/ML/KNeighborsClassifier_%ED%83%80%EC%9D%B4%ED%83%80%EB%8B%89_%EC%83%9D%EC%A1%B4%EC%9E%90_%EC%98%88%EC%B8%A1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

문제 정의 : KNeighborsClassifier 사용하여 타이타닉 생존자(1), 사망(0) 예측

기본 라이브러리 불러오기

In [1]:
import pandas as pd
import seaborn as sns

In [2]:
# 한글 깨짐 방지
import matplotlib as mpl
import matplotlib.pyplot as plt
 
%config InlineBackend.figure_format = 'retina'
 
!apt -qq -y install fonts-nanum
 
import matplotlib.font_manager as fm

fontpath = '/usr/share/fonts/truetype/nanum/NanumBarunGothic.ttf'
font = fm.FontProperties(fname=fontpath, size=9)
plt.rc('font', family='NanumBarunGothic') 
mpl.font_manager._rebuild()

plt.rcParams['axes.unicode_minus'] = False # 마이너스 기호 깨짐 방지

fonts-nanum is already the newest version (20170925-1).
0 upgraded, 0 newly installed, 0 to remove and 39 not upgraded.


[Step1] 데이터 준비 - Seaborn에서 제공하는 titanic data set 가져오기

In [3]:
# load_dataset 함수를 사용하여 데이터 프레임으로 변환
df = sns.load_dataset('titanic')

[Step2] 데이터 탐색 및 데이터 전처리

In [4]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [5]:
# 데이터 자료형 확인
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 15 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    object  
 3   age          714 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     889 non-null    object  
 8   class        891 non-null    category
 9   who          891 non-null    object  
 10  adult_male   891 non-null    bool    
 11  deck         203 non-null    category
 12  embark_town  889 non-null    object  
 13  alive        891 non-null    object  
 14  alone        891 non-null    bool    
dtypes: bool(2), category(2), float64(2), int64(4), object(5)
memory usage: 80.6+ KB


In [6]:
# NaN값이 많은 deck(객실 데크 위치) 열을 삭제, embarked와 내용이 겹치는 embark_town(승선 도시) 열을 삭제

rdf = df.drop(['deck', 'embark_town'], axis=1)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    891 non-null    int64   
 1   pclass      891 non-null    int64   
 2   sex         891 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       891 non-null    int64   
 5   parch       891 non-null    int64   
 6   fare        891 non-null    float64 
 7   embarked    889 non-null    object  
 8   class       891 non-null    category
 9   who         891 non-null    object  
 10  adult_male  891 non-null    bool    
 11  alive       891 non-null    object  
 12  alone       891 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 72.4+ KB


In [8]:
# age 열에 나이 데이터가 없는 모든 행을 삭제 -> 177개 NaN 삭제 (under sampling)
rdf = rdf.dropna(subset=['age'], how='any', axis=0)
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    712 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


In [9]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값 확인 -> 탑승한 도시의 첫글자
most_freq = rdf['embarked'].value_counts(dropna=True).idxmax()
most_freq

'S'

In [11]:
rdf.describe(include='all')

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,alive,alone
count,714.000000,714.000000,714,714.000000,714.000000,714.000000,714.000000,712,714,714,714,714,714
unique,NaN,NaN,2,NaN,NaN,NaN,NaN,3,3,3,2,2,2
top,NaN,NaN,male,NaN,NaN,NaN,NaN,S,Third,man,True,no,True
freq,NaN,NaN,453,NaN,NaN,NaN,NaN,554,355,413,413,424,404
mean,0.406162,2.236695,NaN,29.699118,0.512605,0.431373,34.694514,NaN,NaN,NaN,NaN,NaN,NaN
std,0.491460,0.838250,NaN,14.526497,0.929783,0.853289,52.918930,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,1.000000,NaN,0.420000,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,1.000000,NaN,20.125000,0.000000,0.000000,8.050000,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,2.000000,NaN,28.000000,0.000000,0.000000,15.741700,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.000000,3.000000,NaN,38.000000,1.000000,1.000000,33.375000,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
# embarked 열의 NaN값을 승선도시 중에서 가장 많이 출현한 값으로 치환하기
rdf['embarked'].fillna(most_freq, inplace=True)

In [14]:
rdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 714 entries, 0 to 890
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   survived    714 non-null    int64   
 1   pclass      714 non-null    int64   
 2   sex         714 non-null    object  
 3   age         714 non-null    float64 
 4   sibsp       714 non-null    int64   
 5   parch       714 non-null    int64   
 6   fare        714 non-null    float64 
 7   embarked    714 non-null    object  
 8   class       714 non-null    category
 9   who         714 non-null    object  
 10  adult_male  714 non-null    bool    
 11  alive       714 non-null    object  
 12  alone       714 non-null    bool    
dtypes: bool(2), category(1), float64(2), int64(4), object(4)
memory usage: 63.6+ KB


[Step3] 분석에 사용할 속성 선택

In [25]:
# 분석에 활용할 열(속성)을 선택 (생존 여부,객실 등급, 성별, 나이, 타이타닉에 탑승한 형제 자매 수
# 타이타닉에 탑승한 부모/자녀 수, 탑승한 곳(항구) C=Cherbourg, Q = Queenstown, S = Southhampton)
ndf = rdf[['survived', 'pclass', 'sex', 'sibsp', 'parch', 'embarked']]
ndf.head()

,survived,pclass,sex,sibsp,parch,embarked
0,0,3,male,1,0,S
1,1,1,female,1,0,C
2,1,3,female,0,0,S
3,1,1,female,1,0,S
4,0,3,male,0,0,S


In [26]:
# one-hot 인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# male, female -> [1, 0], [0, 1]
# embarked C, Q, S = [1, 0, 0], [0, 1, 0], [0, 0, 1]

oneshot_sex = pd.get_dummies(ndf['sex'])
oneshot_sex

,female,male
0,0,1
1,1,0
2,1,0
3,1,0
4,0,1
...,...,...
885,1,0
886,0,1
887,1,0
889,0,1


In [27]:
# one-hot 인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# male, female -> [1, 0], [0, 1]

oneshot_sex = pd.get_dummies(ndf['sex'])
ndf = pd.concat([ndf, oneshot_sex], axis=1)

ndf.head()

,survived,pclass,sex,sibsp,parch,embarked,female,male
0,0,3,male,1,0,S,0,1
1,1,1,female,1,0,C,1,0
2,1,3,female,0,0,S,1,0
3,1,1,female,1,0,S,1,0
4,0,3,male,0,0,S,0,1


In [28]:
# one-hot 인코딩 - 범주형 데이터를 모형이 인식할 수 있도록 숫자형으로 변경
# embarked C, Q, S = [1, 0, 0], [0, 1, 0], [0, 0, 1]

oneshot_embarked = pd.get_dummies(ndf['embarked'], prefix='town')
ndf = pd.concat([ndf, oneshot_embarked], axis=1)

ndf.head()

,survived,pclass,sex,sibsp,parch,embarked,female,male,town_C,town_Q,town_S
0,0,3,male,1,0,S,0,1,0,0,1
1,1,1,female,1,0,C,1,0,1,0,0
2,1,3,female,0,0,S,1,0,0,0,1
3,1,1,female,1,0,S,1,0,0,0,1
4,0,3,male,0,0,S,0,1,0,0,1


In [29]:
# 기존 컬럼 삭제
ndf.drop(['sex', 'embarked'], axis=1, inplace=True)
ndf.head()

,survived,pclass,sibsp,parch,female,male,town_C,town_Q,town_S
0,0,3,1,0,0,1,0,0,1
1,1,1,1,0,1,0,1,0,0
2,1,3,0,0,1,0,0,0,1
3,1,1,1,0,1,0,0,0,1
4,0,3,0,0,0,1,0,0,1


[Step4] data set 구분 - train data / test data

In [33]:
# ndf -> X, y (문제집, 정답지)
X = ndf[['pclass', 'sibsp',	'parch', 'female', 'male', 'town_C', 'town_Q', 'town_S']] # 독립변수
y = ndf['survived']

In [45]:
# train data와 test 데이터로 구분 (7:3 비율)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7)

print(X_train.shape)
print(X_test.shape)

(499, 8)
(215, 8)


[Step 5] kNN 분류 모형 - sklearn 사용 

In [52]:
# sklearn 라이브러리에서 kNN 분류 모형 가져오기
from sklearn.neighbors import KNeighborsClassifier

# 모형 객체 생성 (k = 7로 설정)
knn = KNeighborsClassifier(n_neighbors=7)

In [53]:
# train 데이터를 사용하여 모형 학습
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=7, p=2,
                     weights='uniform')

In [54]:
# test data로 y_hat 예측 (분류)
y_hat = knn.predict(X_test)

print(y_hat[0:10])
print(y_test.values[0:10])

[0 1 0 1 0 0 1 1 0 0]
[0 1 0 1 0 0 1 0 0 0]


모델 성능 평가

In [55]:
from sklearn import metrics

# 모형 성능 평가 - 평가지표 계산
knn_report = metrics.classification_report(y_test, y_hat)
print("테스트 평가지표 n_neighbors=7")
print(knn_report)

테스트 평가지표 n_neighbors=5
              precision    recall  f1-score   support

           0       0.78      0.86      0.82       126
           1       0.77      0.66      0.71        89

    accuracy                           0.78       215
   macro avg       0.77      0.76      0.76       215
weighted avg       0.78      0.78      0.77       215



In [56]:
# sklearn을 이용하면 전부 계산해준다
print('accuracy', metrics.accuracy_score(y_test,y_hat))
print('precision', metrics.precision_score(y_test,y_hat))
print('recall', metrics.recall_score(y_test,y_hat))
print('f1', metrics.f1_score(y_test,y_hat))

accuracy 0.7767441860465116
precision 0.7662337662337663
recall 0.6629213483146067
f1 0.7108433734939759


In [57]:
# 이웃의 수에 따른 정확도 저장할 리스트 변수
train_accuracy = []
test_accuracy = []

# 1에서 10까지 n_neighbors를 적용
neighbors_setting = range(1,10)

for n_neighbors in neighbors_setting :

  # 모델 생성
  clf = KNeighborsClassifier(n_neighbors=n_neighbors)
  clf.fit(X_train, y_train)

  # train set 정확도 저장
  train_accuracy.append(clf.score(X_train, y_train))

  # teset set 정확도 저장
  test_accuracy.append(clf.score(X_test, y_test))

# train vs test 정확도 시각화
plt.figure(dpi= 100)
plt.style.use('ggplot')
plt.rc('font', family = 'NAnumBarunGothic')

plt.plot(neighbors_setting, train_accuracy, label='train 정확도')
plt.plot(neighbors_setting, test_accuracy, label='test 정확도')
plt.ylabel('정확도')
plt.xlabel('n_neighbors')
plt.legend()
plt.show()